## Homework 3: Decoding Algorithms

#### This is due at 11.55 pm on Friday, December 7. Please see detailed submission instructions below.  100 points total.

##### How to do this problem set:

- What version of Python should I use? 3.6

- Most of these questions require writing Python code and computing results, and the rest of them have textual answers. To generate the answers, you will have to fill out two supporting files, `vit_starter.py` and `s2s_starter.py`.

- Write all the answers in this ipython notebook. Once you are finished (1) Generate a PDF via (File -> Download As -> PDF) and upload to Gradescope and (2) turn in `vit_starter.py`, `s2s_starter.py`,  and `hw_3.ipynb` on Moodle. If you do the extra credit, repeat these two steps but upload them for the "HW3 Extra Credit" assignment.
  
- **Important:** Check your PDF before you turn it in to gradescope to make sure it exported correctly. If ipython notebook gets confused about your syntax it will sometimes terminate the PDF creation routine early. You are responsible for checking for these errors. If your whole PDF does not print, try running `$jupyter nbconvert --to pdf hw_1.ipynb` to identify and fix any syntax errors that might be causing problems.

- **Important:** When creating your final version of the PDF to hand in, please do a fresh restart and execute every cell in order. Then you'll be sure it's actually right. One convenient way to do this is by clicking `Cell -> Run All` in the notebook menu.
 

##### Academic honesty 

- We will audit the Moodle code from a few dozen students, chosen at random. The audits will check that the code you wrote and turned on Moodle generates the answers you turn in on your Gradescope PDF. If you turn in correct answers on your PDF without code that actually generates those answers, we will consider this a potential case of cheating. See the course page for honesty policies.

- We will also run automatic checks of code on Moodle for plagiarism. Copying code from others is considered a serious case of cheating.

## 1. Viterbi (log-additive form) (40 points)

<img src="1.png">





One HMM chain is shown on the left. In the graph to the right, let $A(y_1,y_2)$ be the log of the transition probability when the model transition from $y_1$ to $y_2$. Similarly, let 
$B_t$ be the log of the emission probability when the model emits $w_t$ at $y_t$.


Let $\vec{y} = (y_1,y_2,...,y_T)$ be a proposed tag sequence for a $T$ length sentence.
The total ***goodness function*** for a solution $\vec{y}$ is

$$ G(\vec{y}) = \sum_{t=1}^{T} B_t(y_t)  + \sum_{t=2}^{T} A(y_{t-1},y_t) $$


**Question 1.1 (40 points)**

Implement additive log-space Viterbi by completing the **viterbi()** function. It takes in tables that represent the $A$ and $B$ functions as input.  We give you an implementation of $G()$ in **vit_starter**, you can check to make sure you understand the data structures, and also the exhaustive decoding algorithm too.  Feel free to add debugging print statements as needed.  The main code runs the exercise example by default.

When debugging, you should make new A and B examples that are very simple. This will test different code paths.  Also you can try the **randomized\_test()** from the starter code.

Look out for negative indexes as a bug.  In python, if you use an index that's too high to be in the list, it throws an error.  But it will silently accept a negative index ... it interprets that as indexing from the right.


In [6]:
%load_ext autoreload
%autoreload 2

# Implement the viterbi() function in vit_starter.py and then run this cell to show your output

from vit_starter import *


if __name__=='__main__':
    A = {(0,0):2, (0,1):1, (1,0):0, (1,1):5}
    Bs= [ [0,1], [0,1], [25,0] ]
    # that's equivalent to: [ {0:0,1:1}, {0:0,1:1}, {0:25,1:0} ]

    y = exhaustive(A, Bs, set([0,1]))
    print("Exhaustive decoding:", y, "score:", goodness_score(y, A, Bs))
    y = viterbi(A, Bs, set([0,1]))
    print("Viterbi    decoding:", y, "score:", goodness_score(y, A, Bs))
    randomized_test()
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Exhaustive decoding: [1, 1, 0] score: 32
Viterbi    decoding: [1, 1, 0] score: 32
output_vocab= range(0, 5)
A= {(0, 0): 0.754159562480547, (0, 1): 0.9018385574069777, (0, 2): 0.28249615353668067, (0, 3): 0.1460954908884845, (0, 4): 0.7384287487931172, (1, 0): 0.8558432044583635, (1, 1): 0.7936448437679272, (1, 2): 0.021188105009149694, (1, 3): 0.6420961088243037, (1, 4): 0.3073858575949885, (2, 0): 0.6691270035594654, (2, 1): 0.6892006617153799, (2, 2): 0.8312566678347595, (2, 3): 0.4212637290163367, (2, 4): 0.8763041996220258, (3, 0): 0.8011310635056264, (3, 1): 0.16710184263936856, (3, 2): 0.8282724788669631, (3, 3): 0.10844842680035838, (3, 4): 0.692039204692729, (4, 0): 0.5904149663437348, (4, 1): 0.582802554072707, (4, 2): 0.9857249650591521, (4, 3): 0.1099386203270647, (4, 4): 0.818056549064896}
Bs= [[0.6648226827793255, 0.18515767899248703, 0.6104719146790614, 0.6335374014974271, 0.1003961905

**Copy and paste the viterbi function that you implemented in `vit_starter.py`.**

In [ ]:
def viterbi(A, B, output_vocab):
    """
    A: a dict of key:value pairs of the form
        {(curtag,nexttag): score}
    with keys for all K^2 possible neighboring combinations,
    and scores are numbers.  We assume they should be used ADDITIVELY, i.e. in log space.
    higher scores mean MORE PREFERRED by the model.

    B: a list where each entry is a dict {tag:score}, so like
    [ {Noun:-1.2, Adj:-3.4}, {Noun:-0.2, Adj:-7.1}, .... ]
    each entry in the list corresponds to each position in the input.

    output_vocab: a set of strings, which is the vocabulary of possible output
    symbols.

    RETURNS:
    the tag sequence yvec with the highest goodness score
    """

    N = len(B)   # length of input sentence

    # viterbi log-prob tables
    V = [{tag:None for tag in output_vocab} for t in range(N)]
    # backpointer tables
    # back[0] could be left empty. it will never be used.
    back = [{tag:None for tag in output_vocab} for t in range(N)]

    # todo implement the main viterbi loop here
    # you may want to handle the t=0 case separately
    if len(B)==0:
        return None

    for i in output_vocab:
        V[0][i]= B[0][i]
    
    for t in range(1,N):
        for i in output_vocab:
            temp =[]
            for j in output_vocab:
                temp.append(V[t-1][j] + A[(j,i)] + B[t][i])
                #print(temp)
            V[t][i]= max(temp)
            back[t][i]= temp.index(V[t][i])
    
    #print(V)
    #print(back)
    # todo implement backtrace also
    res = dict_argmax(V[-1])
    tag_seq= []
    tag_seq.append(res)
    for i in reversed(range(1,N)):
        if(back[i][res]== None):
            continue
        else:
            res= back[i][res]
            tag_seq.insert(0,res)
    #print(tag_seq)
    return tag_seq

## 2. Decoding in seq2seq models (60 points)

In this part of the homework, you will implement both a greedy search and a beam search for a simple sequence-to-sequence model. We provide the code to build and train the network, so all you have to do is write the decoding algorithms. Please make sure PyTorch and numpy are installed properly before working on this section. 

Our sequence-to-sequence model consists of a vanilla RNN encoder and decoder. Given a sequence of characters (e.g., **aabbccdd**), the network is trained to produce the same sequence in reverse order (**ddccbbaa**). While this task is obviously not like machine translation or text summarization, the model and algorithms are the same, so we will use it as a proxy for more complex real-world tasks that require GPUs and huge datasets to train. 

To begin, run the below massive cell to (1) set up the data and vocab, (2) build the network, and (3) train the network. We will train for 50 epochs, which should hopefully take no more than a few minutes on your machine. 


In [7]:
import torch, random
import numpy as np
from s2s_starter import S2S

# set random seeds
torch.manual_seed(1111)
random.seed(1111)
np.random.seed(1111)

# create a dataset of 500 examples using a small vocabulary 
# we will try training on sequences of length 10 and testing on sequences of length 15
# this setup tests whether the model has actually learned an algorithm to reverse its input 
vocab = {'a': 0, 'b': 1, 'c':2, 'd':3, 'e':4}
train_seq_len = 10
test_seq_len = 15
num_train_examples = 500
num_test_examples = 100

train_inputs = torch.LongTensor(num_train_examples, train_seq_len).random_(0, len(vocab)) # random sequences
inv_idx = torch.arange(train_seq_len-1, -1, -1).long()
train_outputs = train_inputs[:, inv_idx] # outputs are just the reverse of the input

test_inputs = torch.LongTensor(num_test_examples, test_seq_len).random_(0, len(vocab))
inv_idx = torch.arange(test_seq_len-1, -1, -1).long()
test_outputs = test_inputs[:, inv_idx]

    
# build the network
net = S2S(20, 50, len(vocab))

# set some parameters for training the network
batch_size = 16
idx_to_w = dict((v,k) for (k,v) in vocab.items())
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
num_epochs = 50

# okay, let's train the network!
for ep in range(num_epochs):
    ep_loss = 0.
    
    for start in range(0, len(train_inputs), batch_size):
        in_batch = train_inputs[start:start+batch_size]
        out_batch = train_outputs[start:start+batch_size]

        preds = net(in_batch, out_batch)        
        batch_loss = loss_fn(preds, out_batch.view(-1))
        ep_loss += batch_loss

        # compute gradients
        optimizer.zero_grad() # reset the gradients from the last batch
        batch_loss.backward() # does backprop!!!
        optimizer.step() # updates parameters using gradients

    if ep % 10 == 0 or ep == num_epochs - 1:
        print('epoch %d, loss %f' % (ep, ep_loss))


epoch 0, loss 50.869789
epoch 10, loss 23.193579
epoch 20, loss 15.435858
epoch 30, loss 11.809247
epoch 40, loss 11.643238
epoch 49, loss 6.552195


You should see the loss go down to about 10. Now, let's try decoding some training sequences. In s2s.py, we have provided a greedy decoding algorithm (greedy_search) which just chooses the argmax character prediction at every time step. Run the below code to execute greedy search on the ***training*** data. You'll see the model's predictions for the first three training sentences, along with the accuracy on the entire training dataset. Accuracy here is defined as the percentage of examples for which we are able to exactly generate the reverse of the input.

In [8]:
net.eval()
probs = []
corr = 0.
total = 0.

for seq_idx, seq in enumerate(train_inputs):

    prob, outseq = net.greedy_search(seq.expand(1, train_seq_len))
    inseq = ' '.join([idx_to_w[c] for c in seq.numpy()])
    outseq = ' '.join([idx_to_w[int(c)] for c in outseq])
    if seq_idx < 3:
        print('inp: %s' % inseq)
        print('out: %s, neg log prob: %f\n' % (outseq, prob))
    if inseq == outseq[::-1]:
        corr += 1
    total += 1
    
print('training accuracy: %d / %d, acc=%0.1f' % (corr, total, 100 * corr / total))

inp: a e a a a b d d e e
out: e e d d e a c b c b, neg log prob: -1.911940

inp: c d b e e c b b e e
out: e e b b c e e b d c, neg log prob: -1.222282

inp: d c a e d e c b b c
out: c b b c e d e a c d, neg log prob: -1.170384

training accuracy: 211 / 500, acc=42.2


### 2.1: Implement beam search (40 points)
These should look pretty decent! Most of the outputs may even be exact reverses of the input. With that said, we can do better. Implement the beam_search in s2s_starter.py and run the following cell. To debug, set the beam_size argument in the cell to 1 and make sure the output sequences and probabilities are identical to the ones produced by greedy search. If you have correctly implemented the function, the final line of output will print a 'success' message. You should also expect to see a higher accuracy than the greedy search! This cell may take a minute or so to run.

In [ ]:
passed_check = True
corr = 0.
total = 0.

for seq_idx, seq in enumerate(train_inputs):
    beams = net.beam_search(seq.expand(1, train_seq_len), beam_size=3)
    inseq = ' '.join([idx_to_w[c] for c in seq.numpy()])
    for beam_idx, beam in enumerate(beams):
        prob = beam[0]
        outseq = beam[1]
        
        if isinstance(prob, torch.Tensor):
            prob = prob.detach().numpy()
        outseq = ' '.join([idx_to_w[int(c)] for c in outseq])
        if seq_idx < 3:
            print('input: %s' % inseq)
            print('beam%d: %s, neg log prob: %f' % (beam_idx, outseq, prob))
            
        if beam_idx == 0:
            if inseq == outseq[::-1]:
                corr += 1
            total += 1

    if seq_idx < 3:
        print('')
    if(net.beam_check(seq.expand(1, train_seq_len)) == False):
        passed_check = False
        
print('training accuracy: %d / %d, acc=%0.1f' % (corr, total, 100 * corr / total))
if passed_check:
    print("success! you've successfully implemented beam search!")
else:
    print("your beam search has bugs, go back and check your code carefully!")

**Tried implementing, not working. Submitted code to moodle.**

### 2.2 (10 pts)
What is the maximum beam size we can use? Why? write answer here

***The maximum beam size we can use is infinite. The greater beam size results in better performance as the multiple candidate sequences increase the likelihood of better matching a target sequence. This increased performance results in a decrease in decoding speed since more paths are kept in memory. So with an infinite beam size, literally all the paths are checked and none of them are pruned making the search identical to breadth first search.***

### 2.3 (10 pts)
Is beam search always guaranteed to find an output whose probability is greater than or equal to the output of greedy search? Why?

***No, because it suffers from exposure bias since it is never exposed to its errors during training. It also tends to have some inconsistencies in loss-evaluation. In problems with an optimal substructure such as shortest path-finding problems, greedy searches are more optimal. For example, for finding three disjoint circles within a given triangle that maximize the total area of the circles, greedy search is optimal over beam search for any number of circles.***

### Extra credit (up to 30 pts)
Before starting the extra credit, please export your current notebook to a PDF and upload it to Gradescope, as you may want to modify previous parts of this homework to solve the extra credit. Once you finish the extra credit, export this notebook again to a new PDF and upload it under the separate "HW3 extra credit" assignment on Gradescope. 

You have a simple goal: achieve over 30% accuracy on the ***test*** set. If you do not reach this number, you will get no points on this extra credit. The below cell runs beam search over the test data and computes the accuracy, which will likely be 0%. Feel free to do anything you like (generate more training data, train for more epochs, use bigger beams, use more powerful models, implement an attention mechanism, etc.) as long as you ***(1) do not hard-code the reverse algorithm anywhere (e.g., return inputs[::-1]) and (2) do not train on input sequences of longer than length 10***. If you succeed, your model will have generalized the reverse algorithm from length <10 to length 15! One thing that might be helpful to do first is to try overfitting your training data (i.e., make its accuracy 100%). Finally ***write what you did to achieve at least 30% accuracy here, or you will receive no points***.

In [ ]:
corr = 0.
total = 0.

for seq in test_inputs:
    beams = net.beam_search(seq.expand(1, test_seq_len), beam_size=3)
    
    inseq = ' '.join([idx_to_w[c] for c in seq.numpy()])
    prob = beams[0][0]
    outseq = beams[0][1]
    prob = prob.detach().numpy()
    outseq = ' '.join([idx_to_w[int(c)] for c in outseq])
    if inseq == outseq[::-1]:
        corr += 1
    total += 1

print('%d / %d, test accuracy is %0.1f' % (corr, total, 100 * corr / total))